# Indicadores visuales del turismo certificado

En este cuaderno exploramos los datasets de empresas turísticas certificadas y resumimos tres perspectivas clave: oferta de alojamientos, especialización de guías y unidades habitacionales disponibles. Cada gráfico sigue una estética inspirada en *The Economist* para comunicar la información con claridad.

In [ ]:
from __future__ import annotations

from pathlib import Path
from typing import Dict
import json

import pandas as pd
import plotly.express as px
import yaml

from funds_etl.extractors.csv_extractor import CSVExtractor

In [ ]:
def resolve_project_root() -> Path:
    cwd = Path.cwd()
    config_here = cwd / "config" / "settings.yaml"
    if config_here.exists():
        return cwd
    config_parent = cwd.parent / "config" / "settings.yaml"
    if config_parent.exists():
        return cwd.parent
    raise FileNotFoundError("No se encontró config/settings.yaml")

PROJECT_ROOT = resolve_project_root()
CONFIG_PATH = PROJECT_ROOT / "config" / "settings.yaml"
CONFIG_DIR = CONFIG_PATH.parent

with CONFIG_PATH.open("r", encoding="utf-8") as stream:
    raw_config = yaml.safe_load(stream)

raw_dir = (CONFIG_DIR / raw_config["paths"]["raw_data_dir"]).resolve()
extractor = CSVExtractor()
datasets: Dict[str, pd.DataFrame] = {}
for dataset_name, info in raw_config["datasets"].items():
    dataset_path = raw_dir / info["filename"]
    datasets[dataset_name] = extractor.extract(dataset_path)

list(datasets.keys())

['regiones_empresas',
 'suma_unidades_habitacionales',
 'tipos_alojamientos_empresas',
 'tipos_clase_guias']

In [3]:
def apply_economist_style(fig, title: str, subtitle: str | None = None):
    fig.update_layout(
        title=dict(text=title if not subtitle else f"{title}<br><sup>{subtitle}</sup>"),
        template="plotly_white",
        plot_bgcolor="#f8fafc",
        paper_bgcolor="#f8fafc",
        font=dict(family="Georgia, serif", color="#1e293b"),
        title_font=dict(size=20, family="Georgia, serif", color="#1e293b"),
        xaxis=dict(
            title=dict(font=dict(family="Georgia, serif")),
            tickfont=dict(family="Georgia, serif"),
            showgrid=True,
            gridcolor="#cbd5f5"
        ),
        yaxis=dict(
            title=dict(font=dict(family="Georgia, serif")),
            tickfont=dict(family="Georgia, serif"),
            showgrid=False
        ),
        legend=dict(font=dict(family="Georgia, serif"))
    )
    return fig

ECONOMIST_COLORS = ["#c1121f", "#f25c54", "#003049", "#8d99ae"]

### 1. Liderazgo de tipos de alojamiento

In [ ]:
lodgings = datasets["tipos_alojamientos_empresas"].copy()

lodgings = lodgings.assign(
    recuento_rut_empresa=pd.to_numeric(lodgings["recuento_rut_empresa"], errors="coerce"),
    resaltado=pd.to_numeric(lodgings.get("resaltado"), errors="coerce")
)

# Clasificar según obligatoriedad del registro (Ley 20.423 + Decreto Nº19)
mandatory_terms = [
    "hotel",
    "hostal",
    "hostel",
    "caba",
    "camping",
    "residencial",
    "apart",
    "lodge",
    "refugio",
]
lodgings["obligatoriedad"] = lodgings["clase"].str.lower().apply(
    lambda c: "Obligatorio" if any(term in c for term in mandatory_terms) else "Voluntario"
)

lodgings = lodgings.sort_values("recuento_rut_empresa", ascending=True)
color_map = {"Obligatorio": "#c1121f", "Voluntario": "#8d99ae"}
fig_lodgings = px.bar(
    lodgings,
    x="recuento_rut_empresa",
    y="clase",
    orientation="h",
    text=lodgings["recuento_rut_empresa"],
    color="obligatoriedad",
    color_discrete_map=color_map,
)
fig_lodgings.update_traces(texttemplate="%{text:,}", textposition="outside")
fig_lodgings.update_layout(
    showlegend=True,
    legend_title="Obligatoriedad",
    xaxis_title="Empresas registradas (RUT)",
    yaxis_title="Clase de alojamiento"
)
apply_economist_style(
    fig_lodgings,
    title="Panorama de alojamientos inscritos al 01/07/25",
    subtitle="Clasificación obligatoria vs voluntaria en el Registro Nacional"
)
fig_lodgings.show()

In [ ]:
output_path = PROJECT_ROOT / "docs" / "lodgings_obligatoriedad.html"
output_path.parent.mkdir(parents=True, exist_ok=True)

fig_lodgings.write_html(output_path, include_plotlyjs="cdn", full_html=True)
print(f"Archivo exportado a {output_path.relative_to(PROJECT_ROOT)}")

### 2. Especialización de guías

In [6]:
guides = datasets["tipos_clase_guias"].copy()
guides["porcentaje_empresas"] = pd.to_numeric(
    guides["porcentaje_empresas"], errors="coerce"
) * 100
text_labels = guides["porcentaje_empresas"].map(lambda v: f"{v:.1f}%" if pd.notna(v) else "N/D")
fig_guides = px.bar(
    guides.sort_values("porcentaje_empresas"),
    x="porcentaje_empresas",
    y="clase",
    text=text_labels,
    orientation="h",
    color="clase",
    color_discrete_sequence=ECONOMIST_COLORS
)
fig_guides.update_traces(textposition="outside")
fig_guides.update_layout(showlegend=False, xaxis_title="Participación (%)")
apply_economist_style(
    fig_guides,
    title="Guías generales dominan el mercado",
    subtitle="Distribución de clases de guías certificadas"
)
fig_guides.show()

### 3. Capacidad de unidades habitacionales certificadas

In [7]:
units = datasets["suma_unidades_habitacionales"].copy()
units["suma_unidades"] = units["suma_unidades"]
fig_units = px.bar(
    units,
    x="categoria_unidad",
    y="suma_unidades",
    text=units["suma_unidades"].map(lambda v: f"{int(v):,}"),
    color="categoria_unidad",
    color_discrete_sequence=ECONOMIST_COLORS
)
fig_units.update_traces(textposition="outside")
fig_units.update_layout(showlegend=False, yaxis_title="Unidades disponibles")
apply_economist_style(
    fig_units,
    title="Unidades habitacionales certificadas",
    subtitle="Total de cabañas reportadas"
)
fig_units.show()

### Determinaciones adicionales

Para profundizar en la calidad del registro, contrastamos cuántas empresas aparecen destacadas dentro de cada categoría y cómo se distribuyen esos sellos especiales frente al universo total.

In [ ]:
lodgings_quality = lodgings.assign(
    resaltado=pd.to_numeric(lodgings["resaltado"], errors="coerce"),
    recuento_rut_empresa=pd.to_numeric(lodgings["recuento_rut_empresa"], errors="coerce")
)
lodgings_quality["share_resaltado"] = (
    lodgings_quality["resaltado"] / lodgings_quality["recuento_rut_empresa"]
).fillna(0)

highlight_focus = lodgings_quality.sort_values("share_resaltado", ascending=False).head(8)
fig_highlight_focus = px.bar(
    highlight_focus.sort_values("share_resaltado"),
    x="share_resaltado",
    y="clase",
    text=highlight_focus["share_resaltado"].map(lambda v: f"{v:.1%}"),
    orientation="h",
    color="clase",
    color_discrete_sequence=ECONOMIST_COLORS
)
fig_highlight_focus.update_traces(textposition="outside")
fig_highlight_focus.update_layout(showlegend=False, xaxis_tickformat=".0%")
apply_economist_style(
    fig_highlight_focus,
    title="¿Dónde se concentran las empresas destacadas?",
    subtitle="Porcentaje de RUT con sello resaltado sobre el total por clase"
)
fig_highlight_focus.show()

highlight_output = PROJECT_ROOT / "docs" / "highlight_focus.html"
highlight_output.parent.mkdir(parents=True, exist_ok=True)
fig_highlight_focus.write_html(highlight_output, include_plotlyjs="cdn", full_html=True)
print(f"Archivo exportado a {highlight_output.relative_to(PROJECT_ROOT)}")

categories = highlight_focus.sort_values("share_resaltado")["clase"].tolist()
percentages = (
    highlight_focus.sort_values("share_resaltado")["share_resaltado"] * 100
).round(2).tolist()

highcharts_path = PROJECT_ROOT / "docs" / "highlight_focus_highcharts.html"
highcharts_template = """<!DOCTYPE html>
<html lang=\"es\">
  <head>
    <meta charset=\"UTF-8\" />
    <meta name=\"viewport\" content=\"width=device-width, initial-scale=1.0\" />
    <title>¿Dónde se concentran las empresas destacadas?</title>
    <script src=\"https://code.highcharts.com/highcharts.js\"></script>
    <script src=\"https://code.highcharts.com/modules/exporting.js\"></script>
    <style>
      :root {
        color-scheme: light dark;
      }
      body {
        margin: 0;
        padding: 2rem;
        font-family: \"Helvetica Neue\", Arial, sans-serif;
        background: #ffffff;
        color: #111827;
        transition: background 0.3s ease, color 0.3s ease;
      }
      body.dark {
        background: #0f172a;
        color: #e2e8f0;
      }
      #container {
        min-height: 520px;
        max-width: 960px;
        margin: 1rem auto 0;
      }
      .toolbar {
        display: flex;
        justify-content: flex-end;
        gap: 0.75rem;
        max-width: 960px;
        margin: 0 auto;
      }
      .toolbar button {
        border: 1px solid transparent;
        border-radius: 999px;
        padding: 0.45rem 1.25rem;
        font-size: 0.95rem;
        cursor: pointer;
        background: #111827;
        color: #f8fafc;
        transition: background 0.2s ease, color 0.2s ease, border-color 0.2s ease;
      }
      body.dark .toolbar button {
        background: #e2e8f0;
        color: #0f172a;
      }
      .toolbar button.secondary {
        background: transparent;
        border-color: currentColor;
      }
    </style>
  </head>
  <body>
    <div class=\"toolbar\">
      <button id=\"toggle-theme\" type=\"button\">Cambiar a tema oscuro</button>
    </div>
    <div id=\"container\"></div>
    <script>
      const categories = __CATEGORIES__;
      const data = __DATA__;

      const themes = {
        light: {
          bodyClass: \"\",
          chart: { backgroundColor: \"transparent\" },
          title: { style: { color: \"#111827\" } },
          subtitle: { style: { color: \"#374151\" } },
          xAxis: {
            labels: { style: { color: \"#111827\" } },
            lineColor: \"#e5e7eb\",
            tickColor: \"#e5e7eb\",
            gridLineColor: \"#f3f4f6\"
          },
          yAxis: {
            labels: { style: { color: \"#111827\" } },
            title: { style: { color: \"#111827\" } },
            gridLineColor: \"#e5e7eb\"
          },
          colors: Highcharts.getOptions().colors,
          tooltip: { backgroundColor: \"rgba(255,255,255,0.95)\", style: { color: \"#111827\" } }
        },
        dark: {
          bodyClass: \"dark\",
          chart: { backgroundColor: \"transparent\" },
          title: { style: { color: \"#f8fafc\" } },
          subtitle: { style: { color: \"#cbd5f5\" } },
          xAxis: {
            labels: { style: { color: \"#e2e8f0\" } },
            lineColor: \"#334155\",
            tickColor: \"#334155\",
            gridLineColor: \"#1f2937\"
          },
          yAxis: {
            labels: { style: { color: \"#e2e8f0\" } },
            title: { style: { color: \"#e2e8f0\" } },
            gridLineColor: \"#1f2937\"
          },
          colors: [\"#fcd34d\", \"#f87171\", \"#60a5fa\", \"#34d399\", \"#c084fc\"],
          tooltip: { backgroundColor: \"rgba(15,23,42,0.95)\", style: { color: \"#f8fafc\" } }
        }
      };

      let currentTheme = \"light\";
      let chartRef = null;

      function buildChart(themeKey) {
        const theme = themes[themeKey];
        document.body.className = theme.bodyClass;
        chartRef = Highcharts.chart(\"container\", {
          chart: {
            type: \"column\",
            backgroundColor: theme.chart.backgroundColor
          },
          colors: theme.colors,
          title: {
            text: \"¿Dónde se concentran las empresas destacadas?\",
            style: theme.title.style
          },
          subtitle: {
            text:
              'Fuente: <a target=\"_blank\" href=\"https://serviciosturisticos.sernatur.cl/\">Registro Nacional de Prestadores Turísticos</a>',
            style: theme.subtitle.style,
            useHTML: true
          },
          xAxis: {
            categories,
            crosshair: true,
            accessibility: {
              description: \"Clases de alojamiento\"
            },
            labels: theme.xAxis.labels,
            lineColor: theme.xAxis.lineColor,
            tickColor: theme.xAxis.tickColor,
            gridLineColor: theme.xAxis.gridLineColor
          },
          yAxis: {
            min: 0,
            title: {
              text: \"% de empresas destacadas\",
              style: theme.yAxis.title.style
            },
            labels: theme.yAxis.labels,
            gridLineColor: theme.yAxis.gridLineColor
          },
          tooltip: {
            valueSuffix: \" %\",
            backgroundColor: theme.tooltip.backgroundColor,
            style: theme.tooltip.style
          },
          plotOptions: {
            column: {
              pointPadding: 0.2,
              borderWidth: 0
            }
          },
          series: [
            {
              name: \"% destacados\",
              data
            }
          ]
        });
      }

      buildChart(currentTheme);

      const toggleBtn = document.getElementById(\"toggle-theme\");
      toggleBtn.addEventListener(\"click\", () => {
        currentTheme = currentTheme === \"light\" ? \"dark\" : \"light\";
        toggleBtn.textContent =
          currentTheme === \"light\" ? \"Cambiar a tema oscuro\" : \"Cambiar a tema claro\";
        buildChart(currentTheme);
      });
    </script>
  </body>
</html>
"""

highcharts_template = (
    highcharts_template
    .replace("__CATEGORIES__", json.dumps(categories, ensure_ascii=False))
    .replace("__DATA__", json.dumps(percentages))
)

highcharts_path.write_text(highcharts_template, encoding="utf-8")
print(f"Archivo exportado a {highcharts_path.relative_to(PROJECT_ROOT)}")

#### Contraste entre oferta total y empresas destacadas
El siguiente gráfico muestra cómo se reparte el total de empresas frente a las que cuentan con sellos resaltados en las categorías con mayor volumen, ayudando a identificar brechas de calidad percibida.

In [ ]:
top_volume = lodgings_quality.nlargest(6, "recuento_rut_empresa").copy()
top_volume["general"] = top_volume["recuento_rut_empresa"] - top_volume["resaltado"].fillna(0)
stack_view = top_volume.melt(
    id_vars=["clase"],
    value_vars=["general", "resaltado"],
    var_name="tipo",
    value_name="empresas"
 )
color_map = {"general": "#8d99ae", "resaltado": "#c1121f"}
fig_stack = px.bar(
    stack_view,
    x="empresas",
    y="clase",
    color="tipo",
    orientation="h",
    color_discrete_map=color_map,
    text=stack_view["empresas"].map(lambda v: f"{int(v):,}")
 )
fig_stack.update_traces(textposition="inside")
fig_stack.update_layout(barmode="stack", legend_title="Tipo", xaxis_title="Empresas")
apply_economist_style(
    fig_stack,
    title="Brecha entre oferta total y sellos destacados",
    subtitle="Top 6 categorías por número de empresas"
 )
fig_stack.show()

stack_path_plotly = PROJECT_ROOT / "docs" / "stack_gap.html"
stack_path_plotly.parent.mkdir(parents=True, exist_ok=True)
fig_stack.write_html(stack_path_plotly, include_plotlyjs="cdn", full_html=True)
print(f"Archivo exportado a {stack_path_plotly.relative_to(PROJECT_ROOT)}")

stack_categories = top_volume["clase"].tolist()
stack_general = [int(x) for x in top_volume["general"].fillna(0).round(0).tolist()]
stack_resaltado = [int(x) for x in top_volume["resaltado"].fillna(0).round(0).tolist()]

stack_highcharts_template = """<!DOCTYPE html>
<html lang=\"es\">
  <head>
    <meta charset=\"UTF-8\" />
    <meta name=\"viewport\" content=\"width=device-width, initial-scale=1.0\" />
    <title>Brecha entre oferta total y sellos destacados</title>
    <script src=\"https://code.highcharts.com/highcharts.js\"></script>
    <script src=\"https://code.highcharts.com/modules/exporting.js\"></script>
    <style>
      :root {
        color-scheme: light dark;
      }
      body {
        margin: 0;
        padding: 2rem;
        font-family: \"Helvetica Neue\", Arial, sans-serif;
        background: #ffffff;
        color: #111827;
        transition: background 0.3s ease, color 0.3s ease;
      }
      body.dark {
        background: #0f172a;
        color: #e2e8f0;
      }
      #container {
        min-height: 520px;
        max-width: 960px;
        margin: 1rem auto 0;
      }
      .toolbar {
        display: flex;
        justify-content: flex-end;
        gap: 0.75rem;
        max-width: 960px;
        margin: 0 auto;
      }
      .toolbar button {
        border: 1px solid transparent;
        border-radius: 999px;
        padding: 0.45rem 1.25rem;
        font-size: 0.95rem;
        cursor: pointer;
        background: #111827;
        color: #f8fafc;
        transition: background 0.2s ease, color 0.2s ease, border-color 0.2s ease;
      }
      body.dark .toolbar button {
        background: #e2e8f0;
        color: #0f172a;
      }
      .toolbar button.secondary {
        background: transparent;
        border-color: currentColor;
      }
    </style>
  </head>
  <body>
    <div class=\"toolbar\">
      <button id=\"toggle-theme\" type=\"button\">Cambiar a tema oscuro</button>
    </div>
    <div id=\"container\"></div>
    <script>
      const categories = __STACK_CATEGORIES__;
      const dataGeneral = __GENERAL_DATA__;
      const dataResaltado = __RESALTADO_DATA__;

      const themes = {
        light: {
          bodyClass: \"\",
          chart: { backgroundColor: \"transparent\" },
          title: { style: { color: \"#111827\" } },
          subtitle: { style: { color: \"#374151\" } },
          xAxis: {
            labels: { style: { color: \"#111827\" } },
            lineColor: \"#e5e7eb\",
            tickColor: \"#e5e7eb\",
            gridLineColor: \"#f3f4f6\"
          },
          yAxis: {
            labels: { style: { color: \"#111827\" } },
            title: { style: { color: \"#111827\" } },
            gridLineColor: \"#e5e7eb\"
          },
          colors: [\"#8d99ae\", \"#c1121f\"],
          tooltip: { backgroundColor: \"rgba(255,255,255,0.95)\", style: { color: \"#111827\" } }
        },
        dark: {
          bodyClass: \"dark\",
          chart: { backgroundColor: \"transparent\" },
          title: { style: { color: \"#f8fafc\" } },
          subtitle: { style: { color: \"#cbd5f5\" } },
          xAxis: {
            labels: { style: { color: \"#e2e8f0\" } },
            lineColor: \"#334155\",
            tickColor: \"#334155\",
            gridLineColor: \"#1f2937\"
          },
          yAxis: {
            labels: { style: { color: \"#e2e8f0\" } },
            title: { style: { color: \"#e2e8f0\" } },
            gridLineColor: \"#1f2937\"
          },
          colors: [\"#94a3b8\", \"#f25c54\"],
          tooltip: { backgroundColor: \"rgba(15,23,42,0.95)\", style: { color: \"#f8fafc\" } }
        }
      };

      let currentTheme = \"light\";
      let chartRef = null;

      function formatValue(value) {
        return Highcharts.numberFormat(value, 0, ',', '.');
      }

      function buildChart(themeKey) {
        const theme = themes[themeKey];
        document.body.className = theme.bodyClass;
        chartRef = Highcharts.chart(\"container\", {
          chart: {
            type: \"column\",
            backgroundColor: theme.chart.backgroundColor
          },
          colors: theme.colors,
          title: {
            text: \"Brecha entre oferta total y sellos destacados\",
            style: theme.title.style
          },
          subtitle: {
            text: \"Top 6 categorías por número de empresas\",
            style: theme.subtitle.style
          },
          xAxis: {
            categories,
            crosshair: true,
            accessibility: {
              description: \"Clases de alojamiento\"
            },
            labels: theme.xAxis.labels,
            lineColor: theme.xAxis.lineColor,
            tickColor: theme.xAxis.tickColor,
            gridLineColor: theme.xAxis.gridLineColor
          },
          yAxis: {
            min: 0,
            title: {
              text: \"Empresas\",
              style: theme.yAxis.title.style
            },
            labels: {
              ...theme.yAxis.labels,
              formatter: function () {
                return formatValue(this.value);
              }
            },
            gridLineColor: theme.yAxis.gridLineColor
          },
          tooltip: {
            shared: true,
            backgroundColor: theme.tooltip.backgroundColor,
            style: theme.tooltip.style,
            formatter: function () {
              const lines = this.points
                .map(point => {
                  const formatted = formatValue(point.y);
                  return `<span style=\"color:${point.color}\">●</span> ${point.series.name}: <b>${formatted} empresas</b>`;
                })
                .join('<br/>');
              return `<span style=\"font-size:0.85rem\">${lines}</span>`;
            }
          },
          plotOptions: {
            column: {
              stacking: 'normal',
              pointPadding: 0.2,
              borderWidth: 0
            }
          },
          series: [
            {
              name: \"General\",
              data: dataGeneral
            },
            {
              name: \"Resaltado\",
              data: dataResaltado
            }
          ]
        });
      }

      function syncButtonLabel(themeKey) {
        toggleBtn.textContent =
          themeKey === \"light\" ? \"Cambiar a tema oscuro\" : \"Cambiar a tema claro\";
      }

      const toggleBtn = document.getElementById(\"toggle-theme\");

      toggleBtn.addEventListener(\"click\", () => {
        currentTheme = currentTheme === \"light\" ? \"dark\" : \"light\";
        syncButtonLabel(currentTheme);
        buildChart(currentTheme);
      });

      syncButtonLabel(currentTheme);
      buildChart(currentTheme);
    </script>
  </body>
</html>
"""

stack_highcharts_template = (
    stack_highcharts_template
    .replace("__STACK_CATEGORIES__", json.dumps(stack_categories, ensure_ascii=False))
    .replace("__GENERAL_DATA__", json.dumps(stack_general))
    .replace("__RESALTADO_DATA__", json.dumps(stack_resaltado))
)

stack_highcharts_path = PROJECT_ROOT / "docs" / "stack_gap_highcharts.html"
stack_highcharts_path.write_text(stack_highcharts_template, encoding="utf-8")
print(f"Archivo exportado a {stack_highcharts_path.relative_to(PROJECT_ROOT)}")

In [ ]:
stack_general = top_volume["general"].fillna(0).round(0).astype(int).tolist()
stack_resaltado = top_volume["resaltado"].fillna(0).round(0).astype(int).tolist()

#### Intensidad de especialización en guías
Para las clases de guías, combinamos la escala absoluta y el peso de los destacados para detectar segmentos con alto potencial de formación especializada.

In [10]:
guides_detail = guides.assign(
    resaltado=pd.to_numeric(guides["resaltado"], errors="coerce"),
    recuento_rut_empresa=pd.to_numeric(guides["recuento_rut_empresa"], errors="coerce")
)
guides_detail["share_resaltado"] = (
    guides_detail["resaltado"] / guides_detail["recuento_rut_empresa"]
).replace([pd.NA, pd.NaT], 0).fillna(0) * 100

fig_guides_detail = px.scatter(
    guides_detail,
    x="recuento_rut_empresa",
    y="share_resaltado",
    size="resaltado",
    text="clase",
    color="clase",
    color_discrete_sequence=ECONOMIST_COLORS,
    labels={"recuento_rut_empresa": "Empresas registradas", "share_resaltado": "% destacados"}
)
fig_guides_detail.update_traces(textposition="top center")
apply_economist_style(
    fig_guides_detail,
    title="Especialización de guías según volumen y sello",
    subtitle="Cada burbuja pondera la cantidad de guías destacados"
)
fig_guides_detail.show()